## Email sender

In [81]:
pip install smtplib

^C
Note: you may need to restart the kernel to use updated packages.


In [114]:
import smtplib, ssl

def sendAlert(emergent_email):
    port = 587  # For starttls
    smtp_server = "smtp.gmail.com"
    sender_email = "hrtech@gmail.com"
    receiver_email = emergent_email
    password = "******"
    message = """\
    Observed abnormal heart signals."""

    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, port) as server:
        server.ehlo()  # Can be omitted
        server.starttls(context=context)
        server.ehlo()  # Can be omitted
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)

## Alert DNN Model

In [ ]:
# Importing Required Lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv1D
import wfdb                            # Package for loading the ecg and annotation
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

In [ ]:
data = '/input/mit-bih-arrhythmia-database/'

In [ ]:
patients = ['100','101','102','103','104','105','106','107',
           '108','109','111','112','113','114','115','116',
           '117','118','119','121','122','123','124','200',
           '201','202','203','205','207','208','209','210',
           '212','213','214','215','217','219','220','221',
           '222','223','228','230','231','232','233','234']


The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.

In [ ]:
df = pd.DataFrame()

# Reading all .atr files 
for patient in patients:
    # Generating filepath for all .atr file names
    file = data + patient
    # Saving annotation object
    annotation = wfdb.rdann(file, 'atr')
    # Extracting symbols from the object
    symbol = annotation.symbol
    # Saving value counts
    values, counts = np.unique(symbol, return_counts=True)
    # Writing data points into dataframe
    df_sub = pd.DataFrame({'symbol':values, 'Counts':counts, 'Patient Number':[patient]*len(counts)})
    # Concatenating all data points  
    df = pd.concat([df, df_sub],axis = 0)
df

In [ ]:
# Abnormal Beat Symbols
abnormal = ['L','R','V','/','A','f','F','j','a','E','J','e','S']

# Normal Beat Symbols
normal = ['N']

df.loc[df.symbol == 'N','category'] = 0
df.loc[df.symbol.isin(abnormal), 'category'] = 1

In [ ]:
def load_ecg(file):    
    # load the ecg
    record = wfdb.rdrecord(file)
    # load the annotation
    annotation = wfdb.rdann(file, 'atr')
    
    # extracting the signal
    p_signal = record.p_signal

    # extracting symbols and annotation index
    atr_sym = annotation.symbol
    atr_sample = annotation.sample
    
    print(p_signal, atr_sym, atr_sample)
    return p_signal, atr_sym, atr_sample
def build_XY(p_signal, df_ann, num_cols, abnormal):
    # this function builds the X,Y matrices for each beat
    # it also returns the original symbols for Y
    
    num_rows = len(df_ann)

    X = np.zeros((num_rows, num_cols))
    Y = np.zeros((num_rows,1))
    sym = []
    
    # keep track of rows
    max_row = 0

    for atr_sample, atr_sym in zip(df_ann.atr_sample.values,df_ann.atr_sym.values):

        left = max([0,(atr_sample - num_sec*fs) ])
        right = min([len(p_signal),(atr_sample + num_sec*fs) ])
        x = p_signal[left: right]
        if len(x) == num_cols:
            X[max_row,:] = x
            Y[max_row,:] = int(atr_sym in abnormal)
            sym.append(atr_sym)
            max_row += 1
    X = X[:max_row,:]
    Y = Y[:max_row,:]
    return X,Y,sym
def make_dataset(pts, num_sec, fs, abnormal):
    # function for making dataset ignoring non-beats
    # input:
    #   pts - list of patients
    #   num_sec = number of seconds to include before and after the beat
    #   fs = frequency
    # output: 
    #   X_all = signal (nbeats , num_sec * fs columns)
    #   Y_all = binary is abnormal (nbeats, 1)
    #   sym_all = beat annotation symbol (nbeats,1)
    
    # initialize numpy arrays
    num_cols = 2*num_sec * fs
    X_all = np.zeros((1,num_cols))
    Y_all = np.zeros((1,1))
    sym_all = []
    
    # list to keep track of number of beats across patients
    max_rows = []
    
    for pt in pts:
        file = data + pt
        
        p_signal, atr_sym, atr_sample = load_ecg(file)
        
        # grab the first signal
        p_signal = p_signal[:,0]
        
        # make df to exclude the nonbeats
        df_ann = pd.DataFrame({'atr_sym':atr_sym,
                              'atr_sample':atr_sample})
        df_ann = df_ann.loc[df_ann.atr_sym.isin(abnormal + ['N'])]
        
        X,Y,sym = build_XY(p_signal,df_ann, num_cols, abnormal)
        sym_all = sym_all+sym
        max_rows.append(X.shape[0])
        X_all = np.append(X_all,X,axis = 0)
        Y_all = np.append(Y_all,Y,axis = 0)
        
    # drop the first zero row
    X_all = X_all[1:,:]
    Y_all = Y_all[1:,:]

    return X_all, Y_all, sym_all

In [ ]:
# Parameter Values
num_sec = 3
fs = 360
# Accessing the fuction and creating a dataset with ECG digital Points
X_all, Y_all, sym_all = make_dataset(patients, num_sec, fs, abnormal)

In [11]:
# Train Test Split
X_train, X_valid, y_train, y_valid = train_test_split(X_all, Y_all, test_size=0.2, random_state=42)

In [32]:
# Relu for activation function and drop out for regularization
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = X_train.shape[1]))
model.add(Dropout(rate = 0.25))
model.add(Dense(1, activation = 'sigmoid'))

In [34]:
model.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

In [35]:
model.fit(X_train, y_train, batch_size = 32, epochs= 10, verbose = 1)

Epoch 1/10
87279/87279 [==============================] - 7s 79us/step - loss: 0.3936 - acc: 0.8320
Epoch 2/10
87279/87279 [==============================] - 6s 73us/step - loss: 0.3720 - acc: 0.8490
Epoch 3/10
87279/87279 [==============================] - 7s 75us/step - loss: 0.3724 - acc: 0.8502
Epoch 4/10
87279/87279 [==============================] - 7s 75us/step - loss: 0.3738 - acc: 0.8496
Epoch 5/10
87279/87279 [==============================] - 6s 74us/step - loss: 0.3707 - acc: 0.8516
Epoch 6/10
87279/87279 [==============================] - 7s 76us/step - loss: 0.3700 - acc: 0.8513
Epoch 7/10
87279/87279 [==============================] - 7s 75us/step - loss: 0.3735 - acc: 0.8489
Epoch 8/10
87279/87279 [==============================] - 7s 75us/step - loss: 0.3725 - acc: 0.8507
Epoch 9/10
87279/87279 [==============================] - 6s 73us/step - loss: 0.3714 - acc: 0.8509
Epoch 10/10
87279/87279 [==============================] - 6s 74us/step - loss: 0.3704 - acc: 0.8502

In [37]:
# Predictions
y_train_preds_dense = model.predict(X_train,verbose = 0)
y_valid_preds_dense = model.predict(X_valid,verbose = 0)

In [79]:
# Threshold Value
threshold = (sum(y_train)/len(y_train))[0]

In [40]:
# Evaluation Metrics
def print_report(y_actual, y_pred, threshold):
    # Function to print evaluation metrics
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > threshold))
    recall = recall_score(y_actual, (y_pred > threshold))
    precision = precision_score(y_actual, (y_pred > threshold))
    specificity = sum((y_pred < threshold) & (y_actual == 0)) /sum(y_actual ==0)
    prevalence = (sum(y_actual)/len(y_actual))
    print('AUC:%.3f'%auc)
    print('Accuracy:%.3f'%accuracy)
    print('Recall:%.3f'%recall)
    print('Precision:%.3f'%precision)
    print('Specificity:%.3f'%specificity)
    print('Prevalence:%.3f'%prevalence)
    print(' ')
    return auc, accuracy, recall, precision, specificity

In [41]:
# Accessing Evaluation Metrics Function
print('On Train Data')
print_report(y_train, y_train_preds_dense, threshold)
print('On Valid Data')
print_report(y_valid, y_valid_preds_dense, threshold)

On Train Data
AUC:0.924
Accuracy:0.879
Recall:0.808
Precision:0.807
Specificity:0.911
Prevalence:0.315
 
On Valid Data
AUC:0.924
Accuracy:0.878
Recall:0.811
Precision:0.803
Specificity:0.909
Prevalence:0.314
 


(0.9242364882145241,
 0.8784601283226398,
 0.8107001900306973,
 0.8034188034188035,
 array([0.9094065]))

In [137]:
#print(model.layers[0].input_shape)
input = np.reshape(input, (1,2160)) #Get input data from wearable devices
res = model.predict(input,verbose=0)
if res > threshold:
    sendAlert(email_receiver)
    print("Abnormal")
else:
    print("Normal")    


Normal
